In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as st
import pingouin as pg
%matplotlib inline 

In [80]:
data = pd.read_csv('Display.csv')

In [81]:
data.head()

,age,fb,display
0,24,No,No
1,26,No,No
2,30,No,No
3,31,No,No
4,36,No,No


In [82]:
print(data['fb'].unique(), data['display'].unique())

['No' 'Yes'] ['No' 'Yes']


In [83]:
data['fbx'] = data['fb'].replace({'No':0, 'Yes':1 })
data['displayx'] = data['display'].replace({'No':0, 'Yes':1})
data['interaction'] = data['fbx'] * data['age']

In [84]:
from statsmodels.tools.tools import add_constant
data = add_constant(data)

### **Model 1**

In [85]:
from statsmodels.genmod import families

In [86]:
m02 = sm.GLM(data['displayx'] , data[['const', 'fbx']],
          family=families.Binomial()).fit(attach_wls=True, atol=1e-10)
print(m02.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:               displayx   No. Observations:                   70
Model:                            GLM   Df Residuals:                       68
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -35.021
Date:                Sat, 04 Jul 2020   Deviance:                       70.042
Time:                        09:18:14   Pearson chi2:                     70.0
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.3437      0.458     -2.931      0.0

In [87]:
m02.aic

74.04165632182608

### **calculating odds Ratio**

In [88]:
(m02.fittedvalues).unique()

array([0.20689655, 0.80487805])

In [89]:
# for the case when fbx= 0

p_1_0 = 0.20689655
p_0_0 = 1-p_1_0

pre_ratio_0 = p_1_0/p_0_0

In [90]:
# for the case when fbx= 1

p_1_1 = 0.80487805
p_0_1 = 1-p_1_1

pre_ratio_1 = p_1_1/p_0_1

In [91]:
odd_ratio = pre_ratio_1/pre_ratio_0
odd_ratio 

15.812500288932297

### **Model 2**

In [92]:
m01 = sm.GLM(data['displayx'] , data[['const', 'fbx', 'age', 'interaction']],
          family=families.Binomial()).fit(attach_wls=True, atol=1e-10)
print(m01.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:               displayx   No. Observations:                   70
Model:                            GLM   Df Residuals:                       66
Model Family:                Binomial   Df Model:                            3
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -33.817
Date:                Sat, 04 Jul 2020   Deviance:                       67.634
Time:                        09:18:15   Pearson chi2:                     69.4
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -2.9482      1.596     -1.847      

In [93]:
infl = m02.get_influence(observed=False)

In [94]:
summary_df = infl.summary_frame()
summary_df.sort_values(by=['standard_resid'], ascending=False)

,dfb_const,dfb_fbx,cooks_d,standard_resid,hat_diag,dffits_internal
16,0.376556,-0.285546,0.070897,1.992546,0.034483,0.376556
38,0.376556,-0.285546,0.070897,1.992546,0.034483,0.376556
59,0.376556,-0.285546,0.070897,1.992546,0.034483,0.376556
5,0.376556,-0.285546,0.070897,1.992546,0.034483,0.376556
8,0.376556,-0.285546,0.070897,1.992546,0.034483,0.376556
...,...,...,...,...,...,...
45,-0.000000,-0.211944,0.052852,-2.056241,0.024390,-0.325120
56,-0.000000,-0.211944,0.052852,-2.056241,0.024390,-0.325120
39,-0.000000,-0.211944,0.052852,-2.056241,0.024390,-0.325120
36,-0.000000,-0.211944,0.052852,-2.056241,0.024390,-0.325120
